# Notebook zur Visualisierung der Daten aus den Fragestellungen 1.1 - 1.4 in einem Dashboard
Dieses Notebook erzeugt ein Dashboard und bietet dann die Möglichkeit, in einer Suchliste eine Gemeinde aus der Schweiz auszuwählen. <br>
Danach werden automatisch folgende Fragestellungen in drei Charts und in einer Karte zu der ausgewählten Gemeinde visualisiert: <br>
1.1. Wie gross ist der Anteil von Gas-, Heizöl- und Elektroheizungen in einer Gemeinde?<br>
1.2. Wie viele EinwohnerInnen und Beschäftigte sind durch hohe Öl- und Gaspreise in einer Gemeinde betroffen?<br>
1.3. Welche Gebiete in einer Gemeinde sind am stärksten betroffen?<br>
1.4. Wie hat sich der Anteil fossiler Heizungen in den letzten 20 Jahren in einer Gemeinde verändert?<br>
<br>
**Datenquellen für die Visualiserung:**<br>
Die Daten für die Charts bzw. Karte werden nächtlich automatisch auf GitHub berechnet und publiziert.<br>
https://github.com/nrohrbach/HeizcheckGemeinde/tree/main/Daten<br>
<br>
**Informatioen zu der eingesetzen Packages:**<br>
Plotly Dash, Open-Source Python-Framwork: https://plotly.com/dash [Stand: 2.10.2022]<br>
GitHub Jupyter-dash: https://github.com/plotly/jupyter-dash [Stand: 2.10.2022] <br>
Leaflet open-source JavaScript library for mobile-friendly interactive maps: https://leafletjs.com [Stand: 12.10.2022]<br>
GitHub dash-leaflet: https://github.com/thedirtyfew/dash-leaflet [Stand: 2.10.2022]<br>
<br>
<br>
<i> CAS Spatial Data Analytics 2022 </i> ¦ <i> Kommunale Übersicht von Heizsystemen und Energieträgern in Wohngebäuden </i> ¦ <i> Stand: 23.10.2022  </i> ¦ <i> Entwickler: Jürg Reist und Nico Rohrbach</i>

### Notebook vorbereiten

In [1]:
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from ipyleaflet import Map
import dash_leaflet as dl
import dash_leaflet.express as dlx

# für lokal auf Windows
#import dash_core_components as dcc 
#import dash_html_components as html

# für Docker auf Linux
from dash import html, dcc 
from dash.dependencies import Input, Output

from dash_extensions.javascript import assign
from csv import DictReader
from datetime import date

In [2]:
# Stylesheet für eine einheitliche Darstellung der Objekte im Browser bzw. mobilen Geräten
stylesheet = ['assets/style.css']

### Lesen alle Heizungen die mit Gas, Heizöl oder Elektrizität geheizt werden - Frage 1.1
Diese Daten werden im Notebook <1.1_Anteil_Heizungen_GHE.ipynb> vorbereitet und werden nun für das Dashboard gelesen<br>
Link zu GitHub: https://nrohrbach.github.io/HeizcheckGemeinde/1.1_Anteil_Heizungen_GHE

In [3]:
# lesen der Daten direkt von GitHub
url = 'https://raw.githubusercontent.com/nrohrbach/HeizcheckGemeinde/main/Daten/Gemeindeliste_1-1.csv'

df1_1 = pd.read_csv(url)

# Datensatz anzeigen
df1_1

,Unnamed: 0,Gemeinde,Energiequelle,Anzahl
0,0,Zürich,Gas,21629
1,1,Zürich,Keine,14170
2,2,Zürich,Heizöl,12696
3,3,Zürich,Weitere,12081
4,4,Winterthur,Unbestimmt,10914
...,...,...,...,...
12346,12346,La Côte-aux-Fées,Gas,1
12347,12347,Hagenbuch,Gas,1
12348,12348,Uebeschi,Gas,1
12349,12349,Muri (AG),Gas,1


In [4]:
# sortierte Liste aller Gemeinden der Schweiz erstellen. Diese wird im Dash bei der Dropdownliste verwendet
lsGemeinden = sorted(list(df1_1.Gemeinde.unique()))

### Anzahl betroffener Personen pro Gemeinde - Frage 1.2
Diese Daten werden im Notebook <1.2_Anzahl_Einwohnende.ipynb> vorbereitet und werden nun für das Dashboard gelesen.<br>
Link zu GitHub: https://nrohrbach.github.io/HeizcheckGemeinde/1.2_Anzahl_Einwohnende

In [5]:
#lesen der Daten direkt von GitHub
url = r'https://raw.githubusercontent.com/nrohrbach/HeizcheckGemeinde/main/Daten/Gemeindeliste_1-2.csv'
dfPax = pd.read_csv(url)
dfPax

,Gemeinde,PaxGas,PaxElektrizität,PaxÖl,PaxKeine,PaxUnbestimmt,PaxWeitere
0,Aadorf,3052.0,374.0,2995.0,2.0,1387.0,1610.0
1,Aarau,8277.0,699.0,8300.0,0.0,2881.0,1607.0
2,Aarberg,797.0,225.0,1987.0,4.0,1282.0,327.0
3,Aarburg,3091.0,223.0,3018.0,8.0,1707.0,607.0
4,Aarwangen,95.0,148.0,2543.0,13.0,1320.0,636.0
...,...,...,...,...,...,...,...
2148,Zwingen,31.0,125.0,1251.0,472.0,0.0,746.0
2149,Zwischbergen,0.0,60.0,1.0,2.0,0.0,15.0
2150,Zäziwil,0.0,108.0,736.0,29.0,319.0,454.0
2151,Zürich,188767.0,391.0,98831.0,25176.0,9402.0,100999.0


In [6]:
# Daten in ein langes Datenformat transformieren
dfPax = pd.melt(dfPax, id_vars='Gemeinde', value_vars=['PaxGas','PaxElektrizität', 'PaxÖl', 'PaxKeine','PaxUnbestimmt','PaxWeitere'])
dfPax.columns  = ['Gemeinde','Energieträger','Anzahl']
dfPax

,Gemeinde,Energieträger,Anzahl
0,Aadorf,PaxGas,3052.0
1,Aarau,PaxGas,8277.0
2,Aarberg,PaxGas,797.0
3,Aarburg,PaxGas,3091.0
4,Aarwangen,PaxGas,95.0
...,...,...,...
12913,Zwingen,PaxWeitere,746.0
12914,Zwischbergen,PaxWeitere,15.0
12915,Zäziwil,PaxWeitere,454.0
12916,Zürich,PaxWeitere,100999.0


In [7]:
# Legendenwerte korrekt ausgeben
Legendenwerte = {"PaxGas" : "Gas",
                "PaxÖl" : "Heizöl",
                "PaxKeine" : "Keine",
                "PaxUnbestimmt" : "Unbestimmt",
                "PaxElektrizität": "Elektrizität",
                "PaxWeitere" : "Weitere"}

dfPax["Energieträger"] = dfPax["Energieträger"].map(Legendenwerte)


### Anteil der Energiequellen an Neubauten in den letzten 100 Jahren - Frage 1.4
Diese Daten werden im Notebook <1.4_Zeitreihe_fossiler_Heizungen.ipynb> vorbereitet und werden nun für das Dashboard gelesen.<br>
Link zu GitHub: https://nrohrbach.github.io/HeizcheckGemeinde/1.4_Zeitreihe_fossiler_Heizungen

In [8]:
#lesen der Daten direkt von GitHub
url = 'https://raw.githubusercontent.com/nrohrbach/HeizcheckGemeinde/main/Daten/Gemeindeliste_1-4.csv'

zeitreihe = pd.read_csv(url)
zeitreihe = zeitreihe.sort_values(by=['Bauperiode'])

zeitreihe['Prozentanteil'] = (100/zeitreihe['GebaeudeTotal'])*zeitreihe['Anzahl']

gemeinde = zeitreihe['Gemeinde'].sort_values()
gemeinde = gemeinde.unique()

zeitreihe['Bauperiode'] = zeitreihe.Bauperiode.replace({
    8011: 'Vor 1919',
    8012: '1919 bis 1945',
    8013: '1946 bis 1960',
    8014: '1961 bis 1970',
    8015: '1971 bis 1980',
    8016: '1981 bis 1985',
    8017: '1986 bis 1990',
    8018: '1991 bis 1995',
    8019: '1996 bis 2000',
    8020: '2001 bis 2005',
    8021: '2006 bis 2010',
    8022: '2011 bis 2015',
    8023: 'ab 2016'   
    })

### GeoJSON aller Gemeinden in der Schweiz mit Koordinaten WGS84 erstellen
Dieses GeoJSON dient dazu, dass die Karte automatisch zur ausgewählten Gemeinde wechselt, sobald eine Gemeinde in der Dropdownliste ausgewählt wurde.

Info zum Datensatz: https://opendata.swiss/de/dataset/amtliches-ortschaftenverzeichnis-mit-postleitzahl-und-perimeter [Stand: 14.10.2022]<br>
Download: CSV , WGS84 Download: https://data.geo.admin.ch/ch.swisstopo-vd.ortschaftenverzeichnis_plz/PLZO_CSV_WGS84.zip [Stand: 14.10.2022]   

In [9]:
# Lesen aller Gemeinden und speichern in eine list of Dictionaries
with open("GWR_Data/CH_Gemeinden.csv", 'r') as f:
    dict_reader = DictReader(f)
    CH_Gemeindeliste = list(dict_reader)

# Erster Eintrag anzeigen       
CH_Gemeindeliste [0]

{'name': 'Aadorf', 'lat': '47.49135943', 'lon': '8.897698233'}

In [10]:
# Generieren eines GeoJSON Fils mit Maker für jede Gemeinde inkl. dem Namen als Tooltip
geojson = dlx.dicts_to_geojson([{**c, **dict(tooltip=c['name'])} for c in CH_Gemeindeliste])

# Estellen der Javascript Funktion für das Filtern nach eines gewählten Gemeindenamens
geojson_filter = assign("function(feature, context){return context.props.hideout == feature.properties.name;}")

### Globale Farbdefinition und Datum ermitteln
Durch diese Definition haben die Energieträger in den Charts immer die gleiche Farbe. Dies macht die Charts besser lesbar und wirken harmonischer. <br>
https://plotly.com/python/builtin-colorscales/ [Stand: 23.10.2022] <br>
https://plotly.com/python/colorscales/ [Stand: 23.10.2022]


In [11]:
# globale Farben für alle Charts definieren
FigColor={'Heizöl':'tan',
           'Gas':'lemonchiffon',
           'Elektrizität':'tomato',
           'Unbestimmt':'rgb(105,105,105)',
           'Weitere':'rgb(128,128,128)',
           'Keine':'rgb(169,169,169)',
           'leer':'rgb(192,192,192)'
          }

In [12]:
# Heutiges Datum ermitteln
today = date.today()
strtoday = 'Die Daten werden täglich aktualisiert. Die letzte erfolgreiche Aktualisierung erfolgte am ' + today.strftime("%d. %B %Y" + '.')

### JupyterDash erstellen und publizieren
Im Dashboard kann man in der Dropdownliste eine Gemeinde auswählen. Es kann auch nach der gewünschten Gemeinde gesucht werden.
Nach einem Klick auf die ausgewählte Gemeinde werden alle vier Charts automatisch aktualisiert. In der Karte erfolgt ein Zoom auf die gerade ausgewählte Gemeinde.


In [13]:
# JupyterDash erstellen
app = JupyterDash(__name__, external_stylesheets=stylesheet)

# Titel im Browser-Tab 
app.title = "Heizenergieträger in Gemeinden"

# Titel und einleitender Text inkl. Quellen
app.layout = html.Div(children=[
    html.Div(children=[
        html.H3(children='Wie gross ist der Anteil von Heizöl-, Gas- und Elektroheizungen in einer Gemeinde?'),
        html.P(children='Ein Drittel des Energieverbrauchs in der Schweiz wird durch Haushalte verursacht.\
                         Ein grosser Teil dieses Energieverbrauchs entsteht durch Gebäudeheizungen.\
                         In der Schweiz werden 60% der Gebäude mit Gas oder Öl beheizt.\
                         Durch den Ukrainekrieg sind die Energiepreise stark gestiegen, insbesondere für Gas und Öl.\
                         Dadurch müssen Haushalte und Betriebe welche mit Öl oder Gas heizen, mit höheren Ausgaben rechnen.\
                         Diese zusätzliche finanzielle Belastung der Bevölkerung ist nicht regelmässig verteilt und trifft nicht alle Regionen\
                         oder Gemeinden im gleichen Ausmass. Darum soll mit dieser Arbeit analysiert werden, wie stark verschiedene Gemeinden\
                         von den steigenden Energiepreisen betroffen sind.'),
        
       ], style={'textAlign': 'left',
           'line-height': '2rem'}),

    ### Dropdownliste für die Gemeinde Auswahl ###
    html.Div(children=[
        html.Label([
        html.P(children='Bitte Gemeinde auswählen / bzw. suchen:', style={'font-weight': 'bold'}),
        dcc.Dropdown(id='Gemeinde-dropdown', 
                     clearable=False,
                     #value=lsGemeinden[0],
                     value='Olten',
                     options=[{'label': c, 'value': c}
                for c in lsGemeinden
            ])
        ])
    ], className="eleven columns", style={'width': '96.5%',
                                          'padding':'1rem',
                                          'marginTop':'1rem',
                                          'marginLeft':'1rem',
                                          'boxShadow': '#e3e3e3 4px 4px 2px',
                                          'border-radius': '10px',
                                          'backgroundColor': 'withe'}),        

             
    ### PiePlot für die ausgewählte Gemeinde - Frage 1.1 ###
    html.Div(children=[
             dcc.Graph(id='anteil')
        ], className="six columns", style={'padding':'1rem',
                                           'marginTop':'1rem',
                                           'marginLeft':'1rem',
                                           'boxShadow': '#e3e3e3 4px 4px 2px',
                                           'border-radius': '10px',
                                           'backgroundColor': 'withe'}),
    
    ### Karte mit WMS vom QGIS-Server und im Hintergund die GeoJSON Marker - Frage 1.3  ###
    html.Div(children=[
           # Beschriftung oberhalb der Karte
           html.P(children='Heatmap der betroffenen Gebiete (Heizöl, Gas und Elektrizität)', 
                           style={'marginLeft':'2.5em',
                                  'font-family': 'Verdana',
                                  'font-size': '1.1em',
                                  'font-weight': 'bold',
                                  'textAlign': 'left',
                                  'color': 'rgb(62,63,95)'}),
           # WMS Karte       
           dl.Map([dl.WMSTileLayer(url="https://wms.rei.st/?SERVICE=WMS", layers="Analyse", format="image/png", transparent=True, minZoom = 8, maxZoom = 17),
           # GeoJSON für die Gemeindesuche in der Karte
           dl.GeoJSON(data=geojson, options=dict(filter=geojson_filter), id="geojson", zoomToBounds=True)       
           ], id='mapscale', zoom=15, className="six columns", style={'width': '97%',
                                                                   'height': '400px',
                                                                   'padding':'1rem',
                                                                   'marginTop':'1rem',
                                                                   'marginBottom':'1rem',
                                                                   'marginLeft':'1rem',
                                                                   'boxShadow': '#e3e3e3 4px 4px 2px',
                                                                   'border-radius': '10px'}),    
        
    ], className="six columns", style={'padding':'1rem',
                                       'marginTop':'1rem',
                                       'marginLeft':'1rem',
                                       'boxShadow': '#e3e3e3 4px 4px 2px',
                                       'border-radius': '10px'}),  
  
    ### Zeitreihe für die ausgewählte Gemeinde - Frage 1.4 ###
    html.Div(children=[
             dcc.Graph(id='zeitreihe')
        ], className="six columns", style={'padding':'1rem',
                                           'marginTop':'1rem',
                                           'marginLeft':'1rem',
                                           'boxShadow': '#e3e3e3 4px 4px 2px',
                                           'border-radius': '10px',
                                           'backgroundColor': 'withe'}),
        
    ### Anzahl betroffener Personen pro Gemeinde - Frage 1.2 ###
    html.Div(children=[
             dcc.Graph(id='personen')
        ], className="six columns", style={'padding':'1rem',
                                           'marginTop':'1rem',
                                           'marginLeft':'1rem',
                                           'boxShadow': '#e3e3e3 4px 4px 2px',
                                           'border-radius': '10px',
                                           'backgroundColor': 'withe'}),
      
    ### Datum letzte Aktualisierung, Dokumentation und Links zu GitHub ###
    html.Div(children=[
        html.P(children=strtoday),
        html.P(children='Dieses Dashboard wurde mit Python erstellt. Alle Sourcen sind auf GitHub publiziert:'),
        html.A(children='https://nrohrbach.github.io/HeizcheckGemeinde', href='https://nrohrbach.github.io/HeizcheckGemeinde',
                         style={'target': '_blank', 'rel': 'noopener noreferrer nofollow', 'text-decoration': 'none'}),
        ],
        className="eleven columns", style={'width': '96.5%',
                                           'textAlign': 'left',
                                           'padding':'1rem',
                                           'marginTop':'1rem',
                                           'marginLeft':'1rem',
                                           'boxShadow': '#e3e3e3 4px 4px 2px',
                                           'border-radius': '10px',
                                           'line-height': '2rem'}
    ),

    ### Fusszeile ###
    html.Div(html.P('Weiterbildung der Hochschule für Architektur, Bau und Geomatik FHNW ¦\
                     CAS Spatial Data Analytics 2022 ¦ Heizenergieträger in Gemeinden ¦\
                     23.11.2022 ¦ Students: Jürg Reist und Nico Rohrbach'),
            className="eleven columns", style={'width': '96.5%',
                                               'textAlign': 'left',
                                               'padding':'1rem',
                                               'marginTop':'1rem',
                                               'marginLeft':'1rem',
                                               'boxShadow': '#e3e3e3 4px 4px 2px',
                                               'border-radius': '10px',
                                               'font-style': 'italic'}
            )
    
], style={'padding': '1rem'})

##################################################################################### 
# Callback und Update Kuchendiagramm von Jürg - Frage 1.1
#####################################################################################   

# Definition der callback Funktion um die Grafik der gerade ausgewählten Gemeinde zu erstellen
@app.callback(
    Output('anteil', 'figure'),
    [Input("Gemeinde-dropdown", "value")]
)

def update_figure1(Gemeinde):
    dfselect1 = df1_1.loc[(df1_1['Gemeinde']==Gemeinde)]
    fig1 = px.pie(dfselect1,
                  values='Anzahl',
                  names='Energiequelle',
                  color='Energiequelle',
                  color_discrete_map=FigColor,
                  hole=.4,
                  title='<b> Anteil der Energieträger pro Gemeinde </b>')
    fig1.update_layout(legend_title_text='Energieträger')
    return fig1


##################################################################################### 
# Callback und Update Balkendiagramm Zeitreihe von Nico - Frage 1.4
##################################################################################### 

# Definition der callback Funktion um die Grafik der gerade ausgewählten Gemeinde zu erstellen
@app.callback(
    Output('zeitreihe', 'figure'),
    [Input("Gemeinde-dropdown", "value")]
)

def update_figure2(Gemeinde):
    dfselect2 = zeitreihe.loc[(zeitreihe['Gemeinde']==Gemeinde)]
    fig2 = px.bar(dfselect2,
                  x="Bauperiode",
                  y="Prozentanteil",
                  color="Energiequelle",
                  color_discrete_map=FigColor,
                  title='<b> Anteil der Energieträger pro Bauperiode </b>')
    fig2.update_layout(legend_title_text='Energieträger')
    return fig2

#####################################################################################   
# Callback und Update Balkendiagramm betroffener Personen von Nico - Frage 1.2
##################################################################################### 

# Definition der callback Funktion um die Grafik der gerade ausgewählten Gemeinde zu erstellen
@app.callback(
    Output('personen', 'figure'),
    [Input("Gemeinde-dropdown", "value")]
)

def update_figure3(Gemeinde):
    dfselect3 = dfPax.loc[(dfPax['Gemeinde']==Gemeinde)]
    fig3 = px.bar(dfselect3,
                  x="Energieträger",
                  y= "Anzahl",
                  color="Energieträger",
                  color_discrete_map=FigColor,
                  title='<b> Anteil betroffener Personen pro Energieträger </b>')
    return fig3


##################################################################################### 
# Callback für das Setzen der Zoomstufe in der Karte von Jürg - Frage 1.3
##################################################################################### 

# Setzt die Zoomstufe auf fix auf 15 wenn eine neue Gemeinde ausgewählt wird
@app.callback(
    Output('mapscale', 'zoom'),
    [Input("Gemeinde-dropdown", "value")]
)

def func(viewport):
        viewport = 15
        return viewport

#####################################################################################    
# Callback um die Karte der gerade ausgewählten Gemeinde zu aktualisieren - Frage 1.3
##################################################################################### 

# Suchen der Gemeinde im GeoJSON File und Karte zu diesem Marker verschieben
# Dies könnte auch als normaler callback aufegrufen werden. Ist aber so performanter
app.clientside_callback("function(x){return x;}", Output("geojson", "hideout"), Input("Gemeinde-dropdown", "value"))


# Starten der App, um das Resultat auf https://dash.rei.st anzuzeigen
# Um die App im Internet zu publizieren ist es wichtig, dass der host=0.0.0.0 gesetzt wird!
if __name__ == '__main__':
      app.run_server(mode='external', host='0.0.0.0', port=8050, debug=False)


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8050
 * Running on http://172.17.0.5:8050
Press CTRL+C to quit
127.0.0.1 - - [23/Nov/2022 16:02:49] "GET /_alive_d5f17afa-7dfe-4d95-ab11-859eaf5cab0e HTTP/1.1" 200 -


Dash app running on http://0.0.0.0:8050/
